In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Original photos are 512 X 384 with RGB channels.

## Using ImageDataGenerator to prepare the images

In [15]:
batch_size = 32
#target_size

train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../data/images/train',
                                                    color_mode='rgb',
                                                    target_size=(128,96),
                                                    batch_size=batch_size,
                                                    class_mode="categorical",
                                                    shuffle=True)

validation_generator = test_datagen.flow_from_directory('../data/images/holdout',
                                                       color_mode='rgb',
                                                       target_size=(128,96),
                                                       batch_size=batch_size,
                                                       class_mode='categorical')



Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [17]:
next(train_generator)[0].shape

(32, 128, 96, 3)

In [16]:
model = keras.models.Sequential([
    keras.layers.Conv2D(64, 7, activation='relu', padding='same', input_shape=[128,96,3]),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.Conv2D(128, 3, activation='relu', padding='same'),
    keras.layers.MaxPooling2D(2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [19]:
history = model.fit(train_generator, 
                    epochs=50, 
                    validation_data=validation_generator, 
                    validation_steps= 20//batch_size)

score = model.evaluate(validation_generator, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
72/72 [==============================] - 123s 2s/step - loss: 1.7985 - accuracy: 0.1948
Epoch 2/50
72/72 [==============================] - 117s 2s/step - loss: 1.7527 - accuracy: 0.2195
Epoch 3/50
72/72 [==============================] - 119s 2s/step - loss: 1.7411 - accuracy: 0.2209
Epoch 4/50
72/72 [==============================] - 122s 2s/step - loss: 1.7328 - accuracy: 0.2282
Epoch 5/50
72/72 [==============================] - 117s 2s/step - loss: 1.7413 - accuracy: 0.2106
Epoch 6/50
72/72 [==============================] - 119s 2s/step - loss: 1.7424 - accuracy: 0.2264
Epoch 7/50
72/72 [==============================] - 116s 2s/step - loss: 1.7295 - accuracy: 0.2420
Epoch 8/50
72/72 [==============================] - 116s 2s/step - loss: 1.7276 - accuracy: 0.2348
Epoch 9/50
72/72 [==============================] - 128s 2s/step - loss: 1.7337 - accuracy: 0.2285
Epoch 10/50
72/72 [==============================] - 122s 2s/step - loss: 1.7249 - accuracy: 0.2477
Epoch 11/

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 128, 96, 64)       9472      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 48, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 48, 64)        36928     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 24, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 24, 128)       73856     
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 24, 128)      

In [20]:
model.save('model1.h5')